In [149]:
import os
import numpy as np
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats as st

In [122]:
os.listdir(data_dir)

['OLD-test-fixed-lead-time-event-G0H3-201804202100-60min-forecast.npy',
 'OLD-test-fixed-lead-time-event-G2H12-201509071500-60min-forecast.npy',
 'OLD-test-fixed-lead-time-event-G4H12-202304231500-60min-forecast.npy',
 'test-fixed-lead-time-event-G0H3-201804202100-120min-forecast.npy',
 'test-fixed-lead-time-event-G0H3-201804202100-120min-original.npy',
 'test-fixed-lead-time-event-G0H3-201804202100-120min-scatter.pdf',
 'test-fixed-lead-time-event-G0H3-201804202100-120min.mp4',
 'test-fixed-lead-time-event-G0H3-201804202100-60min-forecast.npy',
 'test-fixed-lead-time-event-G0H3-201804202100-60min-original.npy',
 'test-fixed-lead-time-event-G0H3-201804202100-60min-scatter.pdf',
 'test-fixed-lead-time-event-G0H3-201804202100-60min.mp4',
 'test-fixed-lead-time-event-G2H12-201509071500-120min-forecast.npy',
 'test-fixed-lead-time-event-G2H12-201509071500-120min-original.npy',
 'test-fixed-lead-time-event-G2H12-201509071500-120min-scatter.pdf',
 'test-fixed-lead-time-event-G2H12-2015090715

In [153]:
mode([1,2,3,4,4])

np.int64(4)

In [ ]:
import os
import numpy as np
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats as st

model_id = "IonCastGNN"
# model_id = "IonCastLSTM"
# model_id = "IonCastSFNO"
event_id = "G0H3-201804202100"
# event_id = "G2H12-201509071500"
event_id = "G4H12-202304231500"
data_dir = f"/mnt/ionosphere-experiments/20250815-presentation/{model_id}"
event_f_npy = f"test-long-horizon-event-{event_id}-forecast.npy"
event_o_npy = f"test-long-horizon-event-{event_id}-original.npy"

fname_f = os.path.join(data_dir, event_f_npy)
fname_o = os.path.join(data_dir, event_o_npy)
forecast = np.load(fname_f)
original = np.load(fname_o)
T, H, W = forecast.shape
delta_minutes = 15

minutes_range = np.arange(1, T+1) * delta_minutes + delta_minutes
timestamps_unflattened = np.broadcast_to(minutes_range[:, None, None] , (T, H, W))

n_pts = 360
step = 360 // n_pts
# forecast = forecast[:, 90, 0:-1:step]
# original = original[:, 90, 0:-1:step]
# timestamps_unflattened = timestamps_unflattened[:, 90, 0:-1:step]


timestamps = timestamps_unflattened.flatten()
gt = original.flatten()
pred = forecast.flatten()
print(minutes_range.shape, timestamps.shape, gt.shape, pred.shape)

(48,) (3110400,) (3110400,) (3110400,)


In [110]:
def plot_scatter_with_hist(gt, pred, file_name, timestamps=None, max_points=3000, title=None):
    """
    Create a scatter plot with marginal histograms comparing ground truth vs predicted values.
    
    Parameters:
        gt (np.ndarray): Ground truth values (1D array, flattened across all pixels & frames)
        pred (np.ndarray): Predicted values (1D array, flattened across all pixels & frames)  
        file_name (str): Path to save the plot
        max_points (int): Maximum number of points to plot for performance
        title (str): Title for the plot
    """
    print(f'Saving scatter plot to {file_name}')
    
    # Sample data for performance if needed
    n_total = len(gt)
    if n_total > max_points:
        random.seed(42)  # For reproducible sampling
        sample_indices = random.sample(range(n_total), max_points)
        gt_sampled = gt[sample_indices]
        pred_sampled = pred[sample_indices]
        if timestamps is not None:
            ts_sampled = timestamps[sample_indices]
    else:
        gt_sampled = gt
        pred_sampled = pred
        if timestamps is not None:
            ts_sampled = timestamps
    # Create DataFrame for seaborn
    df = pd.DataFrame({'Ground Truth (TECU)': gt_sampled, 'Predicted (TECU)': pred_sampled})
    if timestamps is not None:
        df["Timestamps"] = ts_sampled
        # Use seaborn jointplot - much faster and cleaner
        g = sns.jointplot(
            data=df, x='Ground Truth (TECU)', y='Predicted (TECU)',
            kind='scatter', alpha=0.4, height=8, marginal_kws=dict(bins=50, fill=True), hue="Timestamps"
        )
    else:
        # Use seaborn jointplot - much faster and cleaner
        g = sns.jointplot(
            data=df, x='Ground Truth (TECU)', y='Predicted (TECU)',
            kind='scatter', alpha=0.4, height=8, marginal_kws=dict(bins=50, fill=True)
        )
    
    # Add light grid for better readability
    g.ax_joint.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)
    
    # Add reference line (perfect prediction) - subtle styling
    max_val = max(np.max(gt_sampled), np.max(pred_sampled))
    min_val = min(np.min(gt_sampled), np.min(pred_sampled))
    g.ax_joint.plot([min_val, max_val], [min_val, max_val], 
                   color='black', linestyle='-', linewidth=1.5, alpha=0.7, 
                   label='Perfect Prediction')
    g.ax_joint.legend()
    
    # Add correlation coefficient (compute on sampled data for speed)
    corr = np.corrcoef(gt_sampled, pred_sampled)[0, 1]
    g.ax_joint.text(0.05, 0.95, f'R = {corr:.3f}', transform=g.ax_joint.transAxes, 
                   bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Add title if provided
    if title:
        # Adjust figure spacing to accommodate title
        g.fig.subplots_adjust(top=0.9)  # Make room for title
        g.fig.suptitle(title, fontsize=12, y=0.95)
    
    plt.savefig(file_name, dpi=150, bbox_inches='tight')
    plt.close()

In [26]:
def plot_scatter_with_hist(gt, pred, file_name, timestamps=None, max_points=3000, title=None):
    """
    Create a scatter plot with marginal histograms comparing ground truth vs predicted values.
    Colors points by categorical timestamps if provided.
    """
    print(f'Saving scatter plot to {file_name}')
    
    # Sample data for performance if needed
    n_total = len(gt)
    if n_total > max_points:
        np.random.seed(42)  # reproducibility
        sample_indices = np.random.choice(n_total, size=max_points, replace=False)
        gt_sampled = gt[sample_indices]
        pred_sampled = pred[sample_indices]
        ts_sampled = timestamps[sample_indices] if timestamps is not None else None
    else:
        gt_sampled = gt
        pred_sampled = pred
        ts_sampled = timestamps if timestamps is not None else None

    # Build DataFrame
    df = pd.DataFrame({
        "Ground Truth (TECU)": gt_sampled,
        "Predicted (TECU)": pred_sampled
    })
    if ts_sampled is not None:
        df["Timestamps"] = pd.Categorical(ts_sampled)

    # Plot
    if ts_sampled is not None:
        g = sns.jointplot(
            data=df,
            x="Ground Truth (TECU)",
            y="Predicted (TECU)",
            kind="scatter",
            hue="Timestamps",
            palette="Spectral",        # <--- change colormap here
            alpha=0.6,
            height=8,
            marginal_ticks=True,
            marginal_kws=dict(bins=50)  # works with hist only
        )
    else:
        g = sns.jointplot(
            data=df,
            x="Ground Truth (TECU)",
            y="Predicted (TECU)",
            kind="scatter",
            alpha=0.4,
            height=8,
            marginal_ticks=True,
            marginal_kws=dict(bins=50)
        )

    # Add grid
    g.ax_joint.grid(True, alpha=0.3, linestyle="-", linewidth=0.5)

    # Reference line
    max_val = max(np.max(gt_sampled), np.max(pred_sampled))
    min_val = min(np.min(gt_sampled), np.min(pred_sampled))
    g.ax_joint.plot(
        [min_val, max_val], [min_val, max_val],
        color="black", linestyle="-", linewidth=1.5, alpha=0.7,
        label="Perfect Prediction"
    )
    g.ax_joint.legend()

    # Correlation coefficient
    corr = np.corrcoef(gt_sampled, pred_sampled)[0, 1]
    g.ax_joint.text(
        0.05, 0.95, f"R = {corr:.3f}", transform=g.ax_joint.transAxes,
        bbox=dict(boxstyle="round", facecolor="white", alpha=0.8)
    )

    # Title
    if title:
        g.fig.subplots_adjust(top=0.9)
        g.fig.suptitle(title, fontsize=12, y=0.95)

    plt.savefig(file_name, dpi=150, bbox_inches="tight")
    plt.close()


In [115]:
def plot_scatter_with_hist(gt, pred, file_name, timestamps=None, max_points=3000, title=None):
    """
    Create a scatter plot with marginal histograms comparing ground truth vs predicted values.
    Colors points continuously if timestamps are provided.
    
    Parameters:
        gt (np.ndarray): Ground truth values (1D array, flattened across all pixels & frames)
        pred (np.ndarray): Predicted values (1D array, flattened across all pixels & frames)  
        file_name (str): Path to save the plot
        max_points (int): Maximum number of points to plot for performance
        title (str): Title for the plot
    """
    print(f'Saving scatter plot to {file_name}')
    
    # Sample data for performance if needed
    n_total = len(gt)
    if n_total > max_points:
        np.random.seed(42)  # For reproducibility
        sample_indices = np.random.choice(n_total, size=max_points, replace=False)
        gt_sampled = gt[sample_indices]
        pred_sampled = pred[sample_indices]
        ts_sampled = timestamps[sample_indices] if timestamps is not None else None
    else:
        gt_sampled = gt
        pred_sampled = pred
        ts_sampled = timestamps if timestamps is not None else None

    # Create DataFrame for marginals
    df = pd.DataFrame({
        "Ground Truth (TECU)": gt_sampled,
        "Predicted (TECU)": pred_sampled
    })

    # === Case 1: With timestamps (continuous colormap) ===
    if ts_sampled is not None:
        # # Base jointplot for marginals only (no scatter points yet)
        # g = sns.jointplot(
        #     data=df,
        #     x="Ground Truth (TECU)",
        #     y="Predicted (TECU)",
        #     kind="hist",  # hist marginals
        #     height=8,
        #     marginal_kws=dict(bins=50, fill=True)
        # )
        df["Timestamps"] = ts_sampled
        # Base jointplot for marginals only (no 2D hist background)
        g = sns.jointplot(
            data=df,
            x="Ground Truth (TECU)",
            y="Predicted (TECU)",
            kind="hist",   # still needed to trigger jointplot
            alpha=0.0,        # make its scatter invisible
            height=8,
            marginal_kws=dict(bins=T, fill=True),
            hue="Timestamps",
            cmap="jet"
        )

        # Add scatter with continuous colormap
        sc = g.ax_joint.scatter(
            gt_sampled,
            pred_sampled,
            c=ts_sampled,
            cmap="jet",
            alpha=0.4,
            edgecolor="none"
        )
        # Add colorbar
        # plt.colorbar(sc, ax=g.ax_joint, label="Minutes")
        plt.show()
    # === Case 2: No timestamps (normal jointplot with scatter) ===
    else:
        g = sns.jointplot(
            data=df,
            x="Ground Truth (TECU)",
            y="Predicted (TECU)",
            kind="scatter",
            alpha=0.4,
            height=8,
            marginal_kws=dict(bins=50, fill=True)
        )

    # Add light grid
    g.ax_joint.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)

    # Add perfect prediction reference line
    max_val = max(np.max(gt_sampled), np.max(pred_sampled))
    min_val = min(np.min(gt_sampled), np.min(pred_sampled))
    g.ax_joint.plot(
        [min_val, max_val], [min_val, max_val],
        color='black', linestyle='-', linewidth=1.5, alpha=0.7,
        label='Perfect Prediction'
    )
    g.ax_joint.legend()

    # Add correlation coefficient
    corr = np.corrcoef(gt_sampled, pred_sampled)[0, 1]
    g.ax_joint.text(
        0.05, 0.95, f'R = {corr:.3f}', transform=g.ax_joint.transAxes,
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8)
    )

    # Add title if provided
    if title:
        g.fig.subplots_adjust(top=0.9)
        g.fig.suptitle(title, fontsize=12, y=0.95)

    plt.savefig(file_name, dpi=150, bbox_inches='tight')
    plt.close()


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random

def plot_scatter_with_hist(
    gt, pred, file_name, timestamps=None, max_points=3000, title=None,
    plot_kind="hexbin",      # "scatter" | "hexbin" | "hist2d" | "kde"
    bins=50, gridsize=50,    # for hist2d/hexbin
    cmap="viridis",          # colormap
    kde_time_bins=5          # number of time quantile groups for KDE when timestamps is given
):
    """
    Plot gt vs pred with marginal histograms. If `timestamps` is provided, show how the
    relationship varies with time using continuous color (hexbin/hist2d) or time-binned KDE.

    plot_kind:
        - "scatter": plain scatter (fallback if no timestamps)
        - "hexbin":  hexagonal bins; color = count (no ts) or mean timestamp (with ts)
        - "hist2d":  rectangular bins; color = count (no ts) or mean timestamp (with ts)
        - "kde":     smooth density; if ts given, separate KDE contours per time quantile
    """
    print(f"Saving scatter plot to {file_name}")

    # ---- sampling (for speed) ----
    n_total = len(gt)
    if n_total > max_points:
        np.random.seed(42)
        idx = np.random.choice(n_total, size=max_points, replace=False)
        gt_s = gt[idx]
        pred_s = pred[idx]
        ts_s = timestamps[idx] if timestamps is not None else None
    else:
        gt_s = gt
        pred_s = pred
        ts_s = timestamps if timestamps is not None else None

    # Convert timestamps to numeric if present (for continuous coloring/ordering)
    ts_num = None
    if ts_s is not None:
        # Handle datetimes, numerics, or categorical labels uniformly
        try:
            # Try datetime conversion; if it fails, fall back
            ts_pd = pd.to_datetime(ts_s)
            ts_num = (ts_pd - ts_pd.min()).total_seconds()
        except Exception:
            # If already numeric -> use as is; else map categories to ordered codes
            if np.issubdtype(np.asarray(ts_s).dtype, np.number):
                ts_num = np.asarray(ts_s).astype(float)
            else:
                ts_cat = pd.Categorical(ts_s)
                ts_num = ts_cat.codes.astype(float)

    # ---- layout with marginals ----
    g = sns.JointGrid(height=8, x=gt_s, y=pred_s)
    # top / right histograms
    g.ax_marg_x.hist(gt_s, bins=bins, alpha=0.8)
    g.ax_marg_y.hist(pred_s, bins=bins, orientation="horizontal", alpha=0.8)

    # ---- joint plot area ----
    if (plot_kind == "scatter") or (ts_s is None and plot_kind == "kde"):
        # Simple scatter (original behavior when no timestamps)
        g.ax_joint.scatter(gt_s, pred_s, alpha=0.4)
    elif plot_kind == "hexbin":
        if ts_s is None:
            im = g.ax_joint.hexbin(gt_s, pred_s, gridsize=gridsize, cmap=cmap)
            plt.colorbar(im, ax=g.ax_joint, label="Count")
        else:
            mode = lambda x: st.mode(x).mode
            im = g.ax_joint.hexbin(
                gt_s, pred_s,
                C=ts_num,                 # per-point time
                reduce_C_function=mode,  # aggregate to mean time in each hex
                gridsize=gridsize, cmap=cmap
            )
            plt.colorbar(im, ax=g.ax_joint, label="Mean timestamp")
    elif plot_kind == "hist2d":
        # Build rectangular-bin heatmap; if timestamps present, color by mean timestamp
        xedges = np.linspace(gt_s.min(), gt_s.max(), bins + 1)
        yedges = np.linspace(pred_s.min(), pred_s.max(), bins + 1)
        if ts_s is None:
            H, _, _ = np.histogram2d(gt_s, pred_s, bins=[xedges, yedges])
            im = g.ax_joint.pcolormesh(xedges, yedges, H.T, shading="auto", cmap=cmap)
            plt.colorbar(im, ax=g.ax_joint, label="Count")
        else:
            # mean timestamp per bin
            xi = np.clip(np.digitize(gt_s, xedges) - 1, 0, bins - 1)
            yi = np.clip(np.digitize(pred_s, yedges) - 1, 0, bins - 1)
            sum_ts = np.zeros((bins, bins), dtype=float)
            cnt = np.zeros((bins, bins), dtype=int)
            np.add.at(sum_ts, (xi, yi), ts_num)
            np.add.at(cnt, (xi, yi), 1)
            with np.errstate(invalid="ignore"):
                mean_ts = np.where(cnt > 0, sum_ts / cnt, np.nan)
            im = g.ax_joint.pcolormesh(xedges, yedges, mean_ts.T, shading="auto", cmap=cmap)
            cbar = plt.colorbar(im, ax=g.ax_joint)
            cbar.set_label("Mean timestamp")
    elif plot_kind == "kde":
        # Smooth density. If timestamps provided, facet by a few time quantiles.
        if ts_s is None:
            sns.kdeplot(x=gt_s, y=pred_s, fill=True, levels=40, thresh=0.02, cmap=cmap, ax=g.ax_joint)
        else:
            # bin timestamps into quantiles and draw contours
            q = min(kde_time_bins, max(2, len(np.unique(ts_num))))
            bins_q = pd.qcut(ts_num, q=q, labels=False, duplicates="drop")
            palette = sns.color_palette(cmap, n_colors=len(np.unique(bins_q)))
            for b, color in zip(sorted(np.unique(bins_q)), palette):
                mask = bins_q == b
                if mask.sum() < 5:
                    continue
                sns.kdeplot(
                    x=gt_s[mask], y=pred_s[mask],
                    ax=g.ax_joint, levels=8, linewidths=1.2, color=color, fill=False,
                    label=f"Time bin {b+1}/{len(np.unique(bins_q))}"
                )
            g.ax_joint.legend(frameon=True)

    # grid + y=x
    g.ax_joint.grid(True, alpha=0.3, linestyle="-", linewidth=0.5)
    vmin = min(float(np.min(gt_s)), float(np.min(pred_s)))
    vmax = max(float(np.max(gt_s)), float(np.max(pred_s)))
    g.ax_joint.plot([vmin, vmax], [vmin, vmax], color="black", linewidth=1.5, alpha=0.7, label="Perfect Prediction")

    # correlation
    corr = float(np.corrcoef(gt_s, pred_s)[0, 1])
    g.ax_joint.text(0.05, 0.95, f"R = {corr:.3f}", transform=g.ax_joint.transAxes,
                    bbox=dict(boxstyle="round", facecolor="white", alpha=0.8))

    if title:
        g.fig.subplots_adjust(top=0.9)
        g.fig.suptitle(title, fontsize=12, y=0.95)

    plt.savefig(file_name, dpi=150, bbox_inches="tight")
    plt.close()


In [157]:
plot_scatter_with_hist(gt=gt, pred=pred, plot_kind="hexbin", cmap="jet", gridsize=50, max_points=3110400, file_name=f"./{model_id}_{event_id}_hexgrid.pdf", timestamps=timestamps, title=f"{model_id} {event_id}")

Saving scatter plot to ./IonCastGNN_G4H12-202304231500_hexgrid.pdf
